<a href="https://colab.research.google.com/github/LGF1/AndreFerreira/blob/master/ProjectoBigDataAndreFerreira20151076.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Para este projecto, foi pedido aos alunos para aplicarem um algortimo á sua escolha e tentar aplicá-lo inclusive com uma enorme quantia de Data.
Para este projecto foi escolhido o algortimo de MapReduce, contudo não foi possivel aplicá-lo a largas quantias de Data.

### MapReduce


   Map Reduce é um modelo de programação para processamento paralelo escalável.
     Escalável aqui significa que ele pode trabalhar em big data com clusters de computação muito grandes.
     Existem muitas implementações: por ex. Apache Hadoop e Apache Spark.
     Podemos usar o Map-Reduce com qualquer linguagem de programação:
         O Hadoop é escrito em Java
         Spark é escrito em Scala, mas tem uma interface Python.
     Linguagens de programação funcionais como Python ou Scala encaixam-se muito bem com o modelo Map Reduce:
         No entanto, não precisamos de usar programação funcional.
    
   Uma implementação do MapReduce cuidará da funcionalidade de baixo nível para que o user não precise se preocupar com:
         balanceamento de carga
         , E / S de rede
         , otimização de rede e transferência de disco
         , manuseamento de falhas na máquina
         , serialização de dados
         , etc ..
     O modelo é projetado para mover o processamento para onde os dados residem.


### Passos de um MapReduce

1- ETL (Extrair, Transformar e Carregar): Um grande conjunto de dados.
2- Operação Map: extrair algo que o user gosta em cada linha
3-  Shuffle and Sort": alocação de tarefa / nó
4-  Reduzir operação: agregar, resumir, filtrar ou transformar
5-   Escrever resultados.

## WordCount: Map and Reduce

Para este projecto foi usado o MapReduce para fazer um word count de um texto com algum tamanho.

### Map:

Para fazermos os Mapping basicamente iremos transformar o nosso texto de forma a que cada palavra seja contada como 1, ou seja, uma palavra irá aparecer 1 vez no texto

### Reduce:

Já no Reduce, a operação de redução agrupa valores de acordo com sua chave e, em seguida, executa uma dedução em cada "key". Portanto as coleções são particionadas em diferentes unidades de armazenamento. O Map-Reduce irá dobrar os dados de forma a minimizar a cópia de dados no cluster. Dados em diferentes partições são reduzidos separadamente em paralelo.
O resultado final é uma redução dos dados reduzidos em cada partição.
Em python é mais apropriado chamar esta redução de reduceByKey

## Função Multi-Threaded de Mapping

In [0]:
from threading import Thread

def schedule_computation_threaded(f, result, data, threads, i):    
    # Each function evaluation is scheduled on a different core.
    def my_job(): 
        print ("Processing data:", data[i], "... ")
        result[i] = f(data[i])
        print ("Finished job #", i  )  
        print ("Result was", result[i]     )   
    threads[i] = Thread(target=my_job)
    
def my_map_multithreaded(f, data):
    n = len(data)
    result = [None] * n
    threads = [None] * n
    print ("Scheduling jobs.. ")
    for i in range(n):
        schedule_computation_threaded(f, result, data, threads, i)
    print ("Starting jobs.. ")
    for i in range(n):
        threads[i].start()
    print ("Waiting for jobs to finish.. ")
    for i in range(n):
        threads[i].join()
    print ("All done.")
    return result

## Criação das funções groupByKey e reduceByKey

In [0]:
def groupByKey(data):
    result = dict()
    for key, value in data:
        if key in result:
            result[key].append(value)
        else:
            result[key] = [value]
    return result
        
def reduceByKey(f, data):
    key_values = groupByKey(data)
    return map(lambda key: 
                   (key, reduce(f, key_values[key])), 
                       key_values)

## Criação da data que vai ser usada

In [0]:
data = map(lambda x: (x, 1), "Quando tudo estava perdido Jose apareceu e resolveu os problemas usando um truque muito simples persistencia nos resultados Contudo o Jose nao estava preparado para a rejeicao que o esperava Jose tinha sido despedido sem do nem piedade mas ele levantou se e continuou a tentar O caminho parecia complicado mas no fim o Jose saiu vitorioso uma vez que a sua persistencia nunca esteve em questao".split())
data

[('Quando', 1),
 ('tudo', 1),
 ('estava', 1),
 ('perdido', 1),
 ('Jose', 1),
 ('apareceu', 1),
 ('e', 1),
 ('resolveu', 1),
 ('os', 1),
 ('problemas', 1),
 ('usando', 1),
 ('um', 1),
 ('truque', 1),
 ('muito', 1),
 ('simples', 1),
 ('persistencia', 1),
 ('nos', 1),
 ('resultados', 1),
 ('Contudo', 1),
 ('o', 1),
 ('Jose', 1),
 ('nao', 1),
 ('estava', 1),
 ('preparado', 1),
 ('para', 1),
 ('a', 1),
 ('rejeicao', 1),
 ('que', 1),
 ('o', 1),
 ('esperava', 1),
 ('Jose', 1),
 ('tinha', 1),
 ('sido', 1),
 ('despedido', 1),
 ('sem', 1),
 ('do', 1),
 ('nem', 1),
 ('piedade', 1),
 ('mas', 1),
 ('ele', 1),
 ('levantou', 1),
 ('se', 1),
 ('e', 1),
 ('continuou', 1),
 ('a', 1),
 ('tentar', 1),
 ('O', 1),
 ('caminho', 1),
 ('parecia', 1),
 ('complicado', 1),
 ('mas', 1),
 ('no', 1),
 ('fim', 1),
 ('o', 1),
 ('Jose', 1),
 ('saiu', 1),
 ('vitorioso', 1),
 ('uma', 1),
 ('vez', 1),
 ('que', 1),
 ('a', 1),
 ('sua', 1),
 ('persistencia', 1),
 ('nunca', 1),
 ('esteve', 1),
 ('em', 1),
 ('questao', 1)]

### Ao fazermos groupByKey da nossa data, a função irá retornar o número de vezes que certa palavra apareceu na nossa data e vai agroupá-las em grupos

In [0]:
groupByKey(data)

{'Contudo': [1],
 'Jose': [1, 1, 1, 1],
 'O': [1],
 'Quando': [1],
 'a': [1, 1, 1],
 'apareceu': [1],
 'caminho': [1],
 'complicado': [1],
 'continuou': [1],
 'despedido': [1],
 'do': [1],
 'e': [1, 1],
 'ele': [1],
 'em': [1],
 'esperava': [1],
 'estava': [1, 1],
 'esteve': [1],
 'fim': [1],
 'levantou': [1],
 'mas': [1, 1],
 'muito': [1],
 'nao': [1],
 'nem': [1],
 'no': [1],
 'nos': [1],
 'nunca': [1],
 'o': [1, 1, 1],
 'os': [1],
 'para': [1],
 'parecia': [1],
 'perdido': [1],
 'persistencia': [1, 1],
 'piedade': [1],
 'preparado': [1],
 'problemas': [1],
 'que': [1, 1],
 'questao': [1],
 'rejeicao': [1],
 'resolveu': [1],
 'resultados': [1],
 'saiu': [1],
 'se': [1],
 'sem': [1],
 'sido': [1],
 'simples': [1],
 'sua': [1],
 'tentar': [1],
 'tinha': [1],
 'truque': [1],
 'tudo': [1],
 'um': [1],
 'uma': [1],
 'usando': [1],
 'vez': [1],
 'vitorioso': [1]}

### No reduceByKey juntamos as vezes que a palavra apareceu num só número

In [0]:
reduceByKey(lambda x, y: x + y, data)

[('em', 1),
 ('Jose', 4),
 ('perdido', 1),
 ('persistencia', 2),
 ('esperava', 1),
 ('esteve', 1),
 ('questao', 1),
 ('caminho', 1),
 ('piedade', 1),
 ('truque', 1),
 ('tinha', 1),
 ('vitorioso', 1),
 ('complicado', 1),
 ('despedido', 1),
 ('usando', 1),
 ('sem', 1),
 ('nem', 1),
 ('sua', 1),
 ('no', 1),
 ('Quando', 1),
 ('vez', 1),
 ('nao', 1),
 ('ele', 1),
 ('muito', 1),
 ('saiu', 1),
 ('uma', 1),
 ('tudo', 1),
 ('estava', 2),
 ('Contudo', 1),
 ('fim', 1),
 ('do', 1),
 ('mas', 2),
 ('tentar', 1),
 ('para', 1),
 ('preparado', 1),
 ('parecia', 1),
 ('levantou', 1),
 ('rejeicao', 1),
 ('resultados', 1),
 ('apareceu', 1),
 ('O', 1),
 ('simples', 1),
 ('nos', 1),
 ('a', 3),
 ('e', 2),
 ('resolveu', 1),
 ('o', 3),
 ('continuou', 1),
 ('um', 1),
 ('sido', 1),
 ('problemas', 1),
 ('que', 2),
 ('os', 1),
 ('nunca', 1),
 ('se', 1)]

## Paralelização MapReduce

 Podemos facilmente transformar a nossa implementação Map-Reduce numa estrutura paralela e multi-threaded usando a função my_map_multithreaded que definimos anteriormente.
 Isto permitirá executar cálculos de redução de mapa que exploram o processamento paralelo usando vários núcleos num único computador.


In [0]:
def reduceByKey_multithreaded(f, data):
    key_values = groupByKey(data)
    return my_map_multithreaded(
        lambda key: (key, reduce(f, key_values[key])), key_values.keys())

In [0]:
reduceByKey_multithreaded(lambda x, y: x + y, data)

Scheduling jobs.. 
Starting jobs.. 
('Processing data:', 'em', '... ')
('Finished job #', 0)
('Result was', ('em', 1))
('Processing data:', 'Jose', '... ')
('Finished job #', 1)
('Result was', ('Jose', 4))
('Processing data:', 'perdido', '... ')
('Finished job #', 2)
('Processing data:', 'persistencia', '... ')('Processing data:', 'esperava', '... ') ('Result was', ('perdido', 1))

('Processing data:', 'esteve', '... ')
('Finished job #', 3)('Finished job #', 4)
('Finished job #', 5)


('Result was', ('persistencia', 2))('Result was', ('esperava', 1))('Result was', ('esteve', 1))


('Processing data:', 'questao', '... ')
('Processing data:', 'caminho', '... ')('Finished job #', 6)
('Result was', ('questao', 1))

('Finished job #', 7) 
('Processing data:', 'piedade', '... ')('Result was', ('caminho', 1))
('Processing data:', 'truque', '... ')
('Finished job #', 8)

('Result was', ('piedade', 1))
('Finished job #', 9) ('Processing data:', 'tinha', '... ')
('Result was', ('truque', 1))

(

[('em', 1),
 ('Jose', 4),
 ('perdido', 1),
 ('persistencia', 2),
 ('esperava', 1),
 ('esteve', 1),
 ('questao', 1),
 ('caminho', 1),
 ('piedade', 1),
 ('truque', 1),
 ('tinha', 1),
 ('vitorioso', 1),
 ('complicado', 1),
 ('despedido', 1),
 ('usando', 1),
 ('sem', 1),
 ('nem', 1),
 ('sua', 1),
 ('no', 1),
 ('Quando', 1),
 ('vez', 1),
 ('nao', 1),
 ('ele', 1),
 ('muito', 1),
 ('saiu', 1),
 ('uma', 1),
 ('tudo', 1),
 ('estava', 2),
 ('Contudo', 1),
 ('fim', 1),
 ('do', 1),
 ('mas', 2),
 ('tentar', 1),
 ('para', 1),
 ('preparado', 1),
 ('parecia', 1),
 ('levantou', 1),
 ('rejeicao', 1),
 ('resultados', 1),
 ('apareceu', 1),
 ('O', 1),
 ('simples', 1),
 ('nos', 1),
 ('a', 3),
 ('e', 2),
 ('resolveu', 1),
 ('o', 3),
 ('continuou', 1),
 ('um', 1),
 ('sido', 1),
 ('problemas', 1),
 ('que', 2),
 ('os', 1),
 ('nunca', 1),
 ('se', 1)]

## Paralelização do Reduce

Particionamos os dados em subconjuntos aproximadamente iguais.

Em seguida, reduzimos cada subconjunto independentemente num núcleo separado.

Os resultados podem ser combinados em uma etapa final de redução.

In [0]:
def split_data(data, split_points):
    partitions = []
    n = 0
    for i in split_points:
        partitions.append(data[n:i])
        n = i
    partitions.append(data[n:])
    return partitions

data = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
partitioned_data = split_data(data, [3])
partitioned_data

[['a', 'b', 'c'], ['d', 'e', 'f', 'g']]

In [0]:
from threading import Thread

def parallel_reduce(f, partitions):

    n = len(partitions)
    results = [None] * n
    threads = [None] * n
    
    def job(i):
        results[i] = reduce(f, partitions[i])

    for i in range(n):
        threads[i] = Thread(target = lambda: job(i))
        threads[i].start()
    
    for i in range(n):
        threads[i].join()
    
    return reduce(f, results)

parallel_reduce(lambda x, y: x + y, partitioned_data)

'abcdefg'